### Importing Libraries

In [2]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras import layers,models,optimizers,losses, metrics
import os
import numpy as np
import matplotlib.pyplot as plt

### Setting up directory paths

In [3]:
base_dir = "/content/drive/MyDrive/Colab Notebooks/Projects/Vehicle Classification"
ds_path = os.path.join(base_dir,"datasets","images")
model_dir_path = os.path.join(base_dir,"models")

In [4]:
os.listdir(ds_path)

['test', 'train', 'val']

In [5]:
test_files,train_ds_path,val_ds_path = [os.path.join(ds_path,dir) for dir in os.listdir(ds_path)]

In [6]:
os.listdir(train_ds_path)

['SUV',
 'bus',
 'family sedan',
 'fire engine',
 'heavy truck',
 'jeep',
 'minibus',
 'racing car',
 'taxi',
 'truck']

In [7]:
os.listdir(val_ds_path)

['SUV',
 'bus',
 'family sedan',
 'fire engine',
 'heavy truck',
 'jeep',
 'minibus',
 'racing car',
 'taxi',
 'truck']

In [8]:
len(os.listdir(test_files))

200

### Defining constants

In [9]:
IMAGE_SIZE = (224,224,)
BATCH_SIZE = 32

### Data Augmentation generator

In [10]:
train_data_generator = preprocessing.image.ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 20,
    brightness_range = (0.5,2.0),
    zoom_range = (0.7,1.3),
    horizontal_flip = True,
    shear_range = 0.2
)
validation_data_generator = preprocessing.image.ImageDataGenerator(
    rescale = 1/255.0
)

### Loading dataset in batches

In [11]:
train_ds = train_data_generator.flow_from_directory(
    train_ds_path,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    shuffle = True
)

Found 1400 images belonging to 10 classes.


In [12]:
val_ds = validation_data_generator.flow_from_directory(
    val_ds_path,
    target_size = IMAGE_SIZE,
    class_mode = "categorical",
    batch_size = 32
)

Found 200 images belonging to 10 classes.


In [13]:
test_ds = utils.image_dataset_from_directory(
    test_files,
    labels = None,
    image_size = IMAGE_SIZE,
    seed = 21
)

Found 200 files belonging to 1 classes.


### Class labels

In [14]:
class_indices = train_ds.class_indices
class_indices

{'SUV': 0,
 'bus': 1,
 'family sedan': 2,
 'fire engine': 3,
 'heavy truck': 4,
 'jeep': 5,
 'minibus': 6,
 'racing car': 7,
 'taxi': 8,
 'truck': 9}

In [15]:
idx2class = {value:key for key,value in class_indices.items()}
idx2class

{0: 'SUV',
 1: 'bus',
 2: 'family sedan',
 3: 'fire engine',
 4: 'heavy truck',
 5: 'jeep',
 6: 'minibus',
 7: 'racing car',
 8: 'taxi',
 9: 'truck'}

In [16]:
num_classes = train_ds.num_classes
num_classes

10

In [17]:
test_images = [os.path.join(test_files,image_name) for image_name in os.listdir(test_files)]
test_images[:2]

['/content/drive/MyDrive/Colab Notebooks/Projects/Vehicle Classification/datasets/images/test/210d3833b9c3b0613e6c03a33b01d13a.jpg',
 '/content/drive/MyDrive/Colab Notebooks/Projects/Vehicle Classification/datasets/images/test/1fa59ad4144c5f668e92b8ab6a0043f8.jpg']

### Loading resnet50 model

In [18]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

In [19]:
resnet50_model = ResNet50(include_top = False,weights="imagenet", input_shape=(224,224,3))

94765736/94765736 [==============================] - 3s 0us/step


In [20]:
for layer in resnet50_model.layers:
  layer.trainable = False

### Defining optimizers and loss functions

In [21]:
learning_rate = optimizers.schedules.PolynomialDecay(
    initial_learning_rate = 0.001,
    decay_steps = 10000,
    end_learning_rate = 0.0001,
    cycle = True,
    name = "learning_rate_scheduler"
)
optimizer = optimizers.Adam(learning_rate = learning_rate)
loss_function = losses.CategoricalCrossentropy()
accu_metric = metrics.CategoricalAccuracy(name="accuracy")

### Vehicle Classifier

In [22]:
class VehicleClassifier(tf.keras.Model):
  def __init__(self,model):
    super().__init__()
    self.model = model

  def call(self,inputs,training=False):
    return self.model(inputs,training=training)

  def compile(self,loss,optimizer,metrics):
    super().compile()
    self.loss = loss
    self.optimizer = optimizer
    self.accu_metric = metrics[0]

  def train_step(self,data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x,training=True)
      loss = self.loss(y,y_pred)

    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss,trainable_vars)
    self.optimizer.apply_gradients(zip(gradients,trainable_vars))

    self.accu_metric.update_state(y,y_pred)
    return {"loss": loss, "accuarcy": self.accu_metric.result()}

  def test_step(self,data):
    x, y = data
    y_pred = self.model(x,training=False)
    loss = self.loss(y,y_pred)
    self.accu_metric.update_state(y,y_pred)
    return {"loss": loss, "accuarcy": self.accu_metric.result()}

  def get_config(self):
    config = {
        "model": self.model.get_config(),
        "loss": self.loss,
        "optimizer": self.optimizer.get_config(),
        "metrics": [metric.name for metric in self.metrics]
    }
    return config

In [23]:
resnet_output = resnet50_model.output
flatten_layer = layers.Flatten()(resnet_output)
dropout_layer = layers.Dropout(0.25)(flatten_layer)
output = layers.Dense(num_classes,activation="softmax")(dropout_layer)
custom_model = tf.keras.Model(inputs=resnet50_model.input,outputs=output)

In [24]:
vehicle_classifier = VehicleClassifier(custom_model)

In [25]:
vehicle_classifier.compile(
    loss = loss_function,
    optimizer = optimizer,
    metrics = [accu_metric]
)

In [26]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(base_dir,"models","checkpoint.model.h5"),
    monitor="val_loss",
    verbose=0,
    mode='min',
    save_best_only=True,
    save_weights_only=True
)
callbacks = [model_checkpoint]

In [27]:
vehicle_classifier.load_weights(os.path.join(model_dir_path,"checkpoint.model.h5"))

ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.

In [ ]:
# vehicle_classifier_history = vehicle_classifier.fit(
#     train_ds,
#     validation_data=val_ds,
#     batch_size=32,
#     epochs=100,
#     callbacks=callbacks)

Epoch 1/100
44/44 [==============================] - 131s 1s/step - loss: 6.6635 - accuarcy: 0.1040 - val_loss: 2.9391 - val_accuarcy: 0.3000
Epoch 2/100
44/44 [==============================] - 52s 1s/step - loss: 2.6257 - accuarcy: 0.2234 - val_loss: 1.2685 - val_accuarcy: 0.3450
Epoch 3/100
44/44 [==============================] - 50s 1s/step - loss: 2.4043 - accuarcy: 0.2968 - val_loss: 1.8913 - val_accuarcy: 0.4400
Epoch 4/100
44/44 [==============================] - 50s 1s/step - loss: 2.4599 - accuarcy: 0.3185 - val_loss: 2.6376 - val_accuarcy: 0.2600
Epoch 5/100
44/44 [==============================] - 49s 1s/step - loss: 3.0277 - accuarcy: 0.2759 - val_loss: 4.2654 - val_accuarcy: 0.4350
Epoch 6/100
44/44 [==============================] - 52s 1s/step - loss: 2.4762 - accuarcy: 0.3504 - val_loss: 1.1645 - val_accuarcy: 0.5300
Epoch 7/100
44/44 [==============================] - 50s 1s/step - loss: 2.3830 - accuarcy: 0.3448 - val_loss: 0.7991 - val_accuarcy: 0.4150
Epoch 8/100


In [1]:
# vehicle_classifier.save_weights(os.path.join(model_dir_path,"best_weights.weights.h5"))

NameError: name 'vehicle_classifier' is not defined

In [ ]:
# vehicle_classifier.load_weights(os.path.join(model_dir_path,"checkpoint_v3_.model.h5"))
# # vehicle_classifier.load_weights(os.path.join(model_dir_path,"model_v3.weights.h5"))

In [ ]:
# def save_dl_model(model_obj,model_name: str):
#   model_obj.save(os.path.join(model_dir_path,f"{model_name}.h5"),save_format="tf")
#   # model_obj.save(os.path.join(model_dir_path,f"{model_name}.keras"),save_format="tf")

# def load_dl_model(model_name: str):
#   model = models.load_model(os.path.join(model_dir_path,model_name))
#   return model

### Testing

In [28]:
def test_model(model):
  plt.figure(figsize=(15,15))
  for i in range(16):
    image_path = test_images[np.random.randint(0,len(test_images))]
    random_image = preprocessing.image.load_img(image_path,target_size=IMAGE_SIZE)
    random_image = preprocessing.image.img_to_array(random_image)
    random_image = np.expand_dims(random_image,axis=0)
    random_image /= 255
    actual_class = ""
    predicted_class = idx2class[np.argmax(model.predict(random_image,verbose=0))]
    plt.subplot(4,4,i+1)
    plt.imshow(random_image[0])
    plt.title(f"Actual class: {actual_class}\n Predicted class: {predicted_class}")
    plt.axis('off')
  plt.tight_layout()
  plt.show()

In [29]:
test_model(vehicle_classifier)